# Analiza rynku nieruchomości w Polsce

In [1]:
!pip3 install pandas numpy matplotlib seaborn plotly scikit-learn openpyxl
!pip3 install "statsmodels==0.14.4" "scipy==1.15.3" yfinance fredapi


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Komórka 2: Import bibliotek i konfiguracja
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Biblioteki do analizy statystycznej
from scipy import stats
from scipy.stats import pearsonr, spearmanr, normaltest
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

# Biblioteki machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Konfiguracja wyświetlania
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Wszystkie biblioteki zostały załadowane pomyślnie!")


✅ Wszystkie biblioteki zostały załadowane pomyślnie!


In [ ]:
import pandas as pd

# Load Excel file
file_path = "ceny_mieszkan.xlsx"
sheet_primary = pd.read_excel(file_path, sheet_name="Rynek pierwotny", header=None)
sheet_secondary = pd.read_excel(file_path, sheet_name="Rynek wtórny", header=None)

# Find the starting row (header) for the data — the row containing 'Kwartał'
def find_data_start(df):
    return df[df.apply(lambda row: row.astype(str).str.contains("Kwartał", case=False).any(), axis=1)].index[0]

# Extract offer and transaction prices from specified columns
def split_offer_trans(df, start_row, offer_cols=18, skip_cols=5, trans_cols=18):
    df_data = df.iloc[start_row:]
    df_data.columns = df_data.iloc[0]  # set headers
    df_data = df_data[1:].reset_index(drop=True)

    offer_df = df_data.iloc[:, :offer_cols]
    trans_df = df_data.iloc[:, offer_cols + skip_cols : offer_cols + skip_cols + trans_cols]
    return offer_df, trans_df

# Apply to both sheets
start_primary = find_data_start(sheet_primary)
primary_offer, primary_trans = split_offer_trans(sheet_primary, start_primary)

start_secondary = find_data_start(sheet_secondary)
secondary_offer, secondary_trans = split_offer_trans(sheet_secondary, start_secondary)

Primary market – offer prices:
6   Kwartał Białystok Bydgoszcz Gdańsk Gdynia** Katowice       Kielce Kraków  \
0  III 2006      3008      2663   3494     4467  3522.59  2834.839339   6816   
1   IV 2006      3117      2811   5252     5284  3734.72  3014.090525   7089   
2    I 2007      3500      3017   5490     5797  3805.67  3241.415228   7192   
3   II 2007      4422      4092   7137     6992  5093.57  3017.729302   7856   
4  III 2007      4651      4466   7440     7238  5697.62  4765.978233   8060   

6 Lublin  Łódź Olsztyn        Opole       Poznań Rzeszów     Szczecin  \
0   3022  2860    2640  3299.151723  3620.845241    2656  3233.916487   
1   3304  3642    2869  3342.843092  4725.658134    3013  3575.024357   
2   3309  4209    2960          NaN  6444.721994    3469  3831.400406   
3   3503  5403    3318  3307.345458      7242.73    3517  4932.747279   
4   4125  5212    3588  4253.276317  5425.817049    4124  5121.045287   

6 Warszawa      Wrocław Zielona Góra  
0     5605